# Flatiron School Data Science Immersive 
# Module 1 Project 
## Multivariate Analysis: Home Prices in King County, WA

##### Author: Valentina C. Fontiveros

<img src="house_banner.png"/>

## Objectives

> To use data science tools to complete a multivariate analysis of factors influencing home prices in King County, WA.

> To demonstrate the skills gained after completing Module 1 of the Data Science Immersive.

## Background

> The dataset* used in this analysis records physical characteristics and prices of approximately twenty thousand housing units sold in King County, WA during 2014 and 2015.

> Seattle, a major tech hub in the world, is located in this county.

> As of 2010, it was the most densely-populated county in Washington.

> *Flatiron School has provided and modified the original dataset.


##### Population Density of Washington State counties.

<img src= "King_County.jpg" width=400 align='left' />

## Data Analysis | OSEMiN Framework 

> On this project, we will use the OSEM Data Science framework.

> OSEM stands for:
    > Obtain 
    > Scrub 
    > Explore 
    > Model 
    > Interpret

### Import Libraries

> The libraries used provide useful tools for data science workflows.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 
from scipy.stats import kurtosis, skew
import folium
from folium import plugins
import statsmodels.formula.api as smf
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy.stats as stats
%matplotlib inline

### Obtain and Sort Data

> Set index to ID
> Ordered by price,zipcode,then ID

In [ ]:
df = pd.read_csv('kc_house_data.csv') # dataset provided by Flatiron School
df.set_index('id', inplace=True)
df.sort_values(by =['price','zipcode','id'], inplace=True, ascending=True)
df.head() #preview data



> There are a number of columns which were not 
visible on the preview display. Let's display all the column names.

In [ ]:
print('There are ' + str(len(df.columns)) + ' columns in the dataframe.')
df.columns


> We see 20 columns of information, describing various attributes of houses in King County.

## Scrub Data 

### House Prices:  
 

#### Price - Distribution

In [ ]:
# generate histogram for price containing all data
prices = df.price/1000
prices.hist(label = 'prices', bins=50,density=True,color='green',alpha=0.4)
prices.plot.kde(label='kde',color = 'green')
plt.title('Sales Price for Houses in King County (2014-2015)')
plt.xlabel('Prices (in 1000s)')
plt.legend()
plt.show()
print ('Skewness =', skew(df.price))
print ('kurtosis =', kurtosis(df.price))


Observations: high positive skew, extremely high + curtosis, long-tailed distribution.

#### Price - Summary Statistics

In [ ]:
# Summary statitics
df.price.describe().apply(lambda x: format(x, 'f'))

Observations: ~ 20,000 samples, high standard deviation / variability

In [ ]:
# Spot outliers
plt.boxplot(df.price/1000)
plt.title('Distribution of House Prices (in 1000s)')

Observations: numerous outliers

#### Price - Store in Dataframe after log normalizing

In [ ]:
# Take the log of price and store in
price_df = pd.DataFrame(np.log(df.price))
price_df.hist(bins=15,color='green',alpha=0.4)
print ('Skewness =', skew(price_df.price))
print ('kurtosis =', kurtosis(price_df.price))

After log normalizing, the skewness and kurtosis improved.

#### Price - Data Type

In [ ]:
price_df.info()

Observation: price is a float, which is appropriate due to the log transformation.

### Features - Store in Dataframe

In [ ]:
# create a dataframe for features
features = df.columns.drop(['price'])
features_df = pd.DataFrame(df[features])
features_df.head(2)

###   Features: Descriptive Statistics

#### Data Types 

In [ ]:
# obtain column info
features_df.info()

Observations:

> There are numerous variables that should be tranformed to categories.

> There are geographical features that would be interesting to display on a map.

> Some variables could be transformed from float to int for efficiency.

### Transform Data Types

Scan features to determine best transformation

In [ ]:
# This function scans features dataframe for unique values

# Even though this output is large, scanning all the data is important
# if we are not familiar with its peculiarities.

def unique(features_df):
    for column in features_df:
        print('Name: ' + column)
        print('-'*10)
        na = features_df[column].isna().sum()
        print('NaN values: ' + str(na))
        print('-'*10)
        unique = features_df[column].unique()

        print(unique)

        print('')

In [ ]:
unique(features_df)

> We can see above than none of the categorical variables are identified as such.
> We need to transform the type for several of these variables.

There are problems with various variables.
Address those one variable at a time below 

#### 1. 'sqft_basement' feature

This feature contains unknown values marked with a "?"

In [ ]:
# Let's explore how many ? entries are in sqft_basement.
value = '?'
count = 0
for row in features_df['sqft_basement']:
    if row == '?':
        count +=1
print('There are ' + str(count) + ' rows with question marks as unknown values.')
print('This represents ' + str(round(count/len(features_df)*100,2)) + ' percent of the data.')

In [ ]:
value = 0
count = 0
for row in features_df['sqft_basement']:
    if row == '?':
        count +=1
print('There are ' + str(count) + ' rows with question marks as unknown values.')
print('This represents ' + str(round(count/len(features_df)*100,2)) + ' percent of the data.')

> There are several ways to transform the unknown ? entries. We could drop these rows or replace them with the median. 

> Personally, I prefer replacing with the median to begin with, which would introduce a neglible amount of noise.

In [ ]:
# Because we can't calculate the mean of the dataset with strings, let's create a dummy
# variable to replace the question mark with first.
value = '?'
dummy = 99999
############# . df.loc[:,"Score1"].median()

features_df.loc[:,'sqft_basement'].replace(value, dummy,inplace=True) 

# Now let's convert the type to int
features_df['sqft_basement'] = features_df['sqft_basement'].map(lambda x: float(x))
features_df['sqft_basement'] = features_df['sqft_basement'].map(lambda x: int(x))
features_df['sqft_basement'].unique()[0:15]

# Now let's replace 99999 with the median
features_df.loc[:,'sqft_basement'].replace(dummy,np.nan, inplace=True) 

# Let's plot a histogram 
features_df['sqft_basement'].hist(bins=30,color='green',alpha=0.4)
print('The median value for this basement area is: ' + str(features_df['sqft_basement'].median()))
print('The mean value for this basement area is: ' + str(features_df['sqft_basement'].mean()))


> Since the median for known basement values is zero, we will replace the unknown ? values
> with zero.


In [ ]:
# Replace original '?' entries with zeros.
median_sqft = features_df['sqft_basement'].median()
features_df['sqft_basement']= features_df['sqft_basement'].replace(np.nan, median_sqft) 
features_df['sqft_basement'].unique()[0:10]
features_df.sqft_basement = features_df.sqft_basement.astype(int)
print('Done replacing "?" with zeros in sqft_basement')


#### 2. 'date' feature:

> Create a column for the year sold based on 'date'. Reformat to integer.
> We will assume that the exact day of the sale is not relevant to our analysis.

In [ ]:
# compute two new features: yr_sold and month_sold
features_df['yr_sold'] = features_df['date'].map(lambda x: x.split('/',3))
features_df['month_sold'] = features_df['yr_sold'].map(lambda x: int(x[0]))
features_df['yr_sold'] = features_df['yr_sold'].map(lambda x: int(x[2]))

print('Years of reported sales:')
print(features_df['yr_sold'].unique())
print('Months of reported sales:')
print(features_df['month_sold'].unique())
#features_df.drop('date', axis=1, inplace=True)

In [ ]:
# Explore how many homes were sold in 2014 and 2015
features_df.yr_sold.hist(color='green',alpha=.4)
plt.xlabel('Year Sold')
plt.ylabel('Count')
plt.title('Number of Homes Sold in 2014, 2015')
plt.show()

> Observations: Sales decreased by nearly 50% in 2015.

In [ ]:
# Find out whether mean/median prices changed from one year to the next.
temp_price = pd.DataFrame(data=df.price)
temp_df = temp_price.join(features_df, how='left')
plt.scatter(temp_df.yr_sold,temp_df.price/1000,color='green',alpha=0.4)
plt.scatter(2014, temp_df.price[temp_df.yr_sold == 2014].mean()/1000, color='indigo',label='Mean Price') 
plt.scatter(2015, temp_df.price[temp_df.yr_sold == 2015].mean()/1000, color='indigo') 
print('Mean price in 2014: ' + str(round(temp_df.price[temp_df.yr_sold==2014].mean(),0)))
print('Mean price in 2015: ' + str(round(temp_df.price[temp_df.yr_sold==2015].mean(),0)))
plt.title('Home Prices in 2014, 2015')
plt.xlabel('Year')
plt.ylabel('Price (in 1000s)')
plt.legend()
plt.show()

> No significant differences are apparent in value of home prices between 2014 and 2015. 
> This probably suggests that this data has been edited for the purposes of this project.
> There is no clear discrimination of price year after year, even though 2014 contains most of the outliers.

In [ ]:
features_df.month_sold.hist(color='green',alpha=.4,bins=12)
plt.title('Home Sales by Month')
plt.xlabel('Month')
plt.ylabel('Sales Count')
plt.show()

> Observations: Sales pick up in the Spring, peak in May and slow down in late Fall and Winter.

In [ ]:
plt.scatter(temp_df.month_sold,temp_df.price/1000,color='green',alpha=0.4,label='Prices')
for x in range(1,12):
    plt.scatter(x, temp_df.price[temp_df.month_sold == x].mean()/1000, color='indigo')
    print(temp_df.price[temp_df.month_sold == x].mean())
plt.scatter(12, temp_df.price[temp_df.month_sold == 12].mean()/1000, color='indigo',label='Mean Sales Price')    
plt.xlabel('Month')
plt.ylabel('Price (in 1000s)')
plt.title('Home Prices by Monthly Sales')
plt.legend()
plt.show()

> Observations: Mean/Median home values do not change month to month

> Conclusion: There is no value in adding these features to the model
> since they do not explain price variations.

In [ ]:
# drop irrelevant information

features_df.drop('yr_sold',axis=1,inplace=True)
features_df.drop('month_sold',axis=1,inplace=True)
features_df.drop('date',axis=1,inplace=True)

### Explore Correlated Variables

Finding correlations between variables is important at this stage to keep
in mind for the later analysis.

In [ ]:
# this function returns a dataframe pinpointed correlated variables
# that exceed a correlation threshhold. 
def find_corr_vars(df, threshhold):
    
    threshhold = threshhold
    corr_vars0 = df.corr()
    print('Numbers of variables tested')
    print(len(corr_vars0.columns))
    corr_vars = (corr_vars0 > threshhold)
    length = len(corr_vars0.columns)
    print('Correlations tested:')
    print(corr_vars.size)
    print('Correlations meeting threshhold:')
    print(corr_vars.sum().sum() - length)

    rows=[]
    columns=[]
    for column in range(0,length):
        for row in range(0,length):
            if corr_vars.iloc[row][column] == True:
                rows.append(row)
                columns.append(column)
    corr1 = []
    corr2 = []
    words = list(corr_vars.columns)
    for index in rows:
        word = words[index]
        corr1.append(word)
    for index2 in columns:
        corr2.append(words[index2])

    corr_df = pd.DataFrame(data={'col1': corr1,'col2':corr2})
    print('')

    print('Correlated Variables over threshhold ' + str(threshhold))

    return corr_df[corr_df.col1 != corr_df.col2]

In [ ]:
threshhold = 0.75 #set correlation threshhold minimum to filter correlations.
# let's find correlated variables above a certain threshhold
feat_price_df = features_df.join(price_df, how='left')
find_corr_vars(feat_price_df, threshhold).sort_values(by='col1',ascending=False)

> Observation: We see here that sqft_living and correlated variables
> have a marked correlation with price even before processing the data

#### Set Categorical Variables / Reformat

#### 1. 'sqft_basement' feature

In [ ]:
plt.scatter(features_df.sqft_basement,price_df.price,color='green',alpha=.4)

In [ ]:
# Initially, this variable was transformed to a category.
# For now, we will leave 'as is'

#### 2. 'yr_sold' feature

In [ ]:
# dropped from analysis due to nil contribution to price variations

#### 3. 'bathrooms' feature

In [ ]:
plt.scatter(features_df.bathrooms,price_df.price,color='green',alpha=.4)

In [ ]:
# Initially, this variable was transformed to a category.
# For now, we will leave 'as is'

#### 4. 'waterfront' feature:

In [ ]:
plt.scatter(temp_df.waterfront,temp_df.price,color='green',alpha=.4)
plt.scatter(0, (temp_df.price[temp_df.waterfront==0]).mean(),color='indigo')
plt.scatter(1, (temp_df.price[temp_df.waterfront==1]).mean(),color='indigo')
plt.xlabel('Waterfront Property?, 0=No, 1=Yes')
plt.ylabel('Price')
plt.title('Waterfront View Impact on Mean Price')
print('Waterfront Mean Price: ' + str(round((temp_df.price[temp_df.waterfront==1]).mean(),0)))
print('No Waterfront Mean Price: ' + str(round((temp_df.price[temp_df.waterfront==0]).mean(),0)))

> Waterfront properties are valued over a million dollars higher!

In [ ]:
# fill NaN values in waterfront and view with zeros.
features_df.waterfront.fillna(0,inplace=True) # 10% of data was missing.
features_df['waterfront'] = features_df['waterfront'].astype(int).astype('category')

In [ ]:
# check that changes did not impact distribution
temp_df.waterfront.fillna(0,inplace=True)
plt.scatter(temp_df.waterfront,temp_df.price,color='green',alpha=.4)
plt.scatter(0, (temp_df.price[temp_df.waterfront==0]).mean(),color='indigo')
plt.scatter(1, (temp_df.price[temp_df.waterfront==1]).mean(),color='indigo')
plt.xlabel('Waterfront Property?, 0=No, 1=Yes')
plt.ylabel('Price')
plt.title('Waterfront View Impact on Mean Price')
print('Waterfront Mean Price: ' + str(round((temp_df.price[temp_df.waterfront==1]).mean(),0)))
print('No Waterfront Mean Price: ' + str(round((temp_df.price[temp_df.waterfront==0]).mean(),0)))

> Filling NaN values did not affect the main conclusion about the waterfront feature.

#### 5. 'view' feature:

In [ ]:
plt.scatter(features_df.view,df.price, color='green', alpha=0.4)
plt.xlabel('Views of Property')
plt.ylabel('Price')
plt.show()


In [ ]:
features_df.drop('view',axis=1,inplace=True)
#features_df.view.fillna(0,inplace=True)       # less than 1% of data was missing.
#features_df['view'] = features_df['view'].astype(int).astype('category')


#### 6. 'bedrooms' feature:

In [ ]:
plt.scatter(features_df.bedrooms,df.price, color='green', alpha=0.4)

In [ ]:
# initially set as category, changed it back to int

#replace type first to INT then to CATEGORY. No variables needed FLOAT type.
features_df['bedrooms'] = features_df['bedrooms'].astype(int)


#### 7. 'floors' feature:

In [ ]:
plt.scatter(features_df.floors,df.price, color='green', alpha=0.4)

In [ ]:
# bin features and set as category
features_df['floors'] = round(features_df['floors'],0).astype(int)

d = {range(0,2):'1', range(2,4):'2plus'}

features_df['floors'] = features_df['floors'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))
features_df['floors'] = features_df['floors'].astype('category')

#### 8. 'conditions' feature

In [ ]:
features_df['condition'].hist(color='green',alpha=.4)

In [ ]:
# bin features and set as category

features_df['condition'] = features_df['condition'].astype(int)
d = {range(0,3):'poor', range(3,4):'average',range(4,6):'above_average'}
features_df['condition'] = features_df['condition'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))

features_df['condition'] = features_df['condition'].astype('category')

#### 9. 'zipcode' feature

In [ ]:
features_df['zipcode'] = features_df['zipcode'].astype(int).astype('category')

# change a format for basement variable so that all sqft are int

#### 10. 'grade' feature:

In [ ]:
plt.scatter(features_df.grade,df.price, color='green', alpha=0.4)

In [ ]:
# changed from category back to int
features_df['grade'] = features_df['grade'].astype(int)

#### 11. 'yr_renovated' feature:

In [ ]:
plt.scatter(features_df.yr_renovated,df.price, color='green', alpha=0.4)

In [ ]:
# Create categories for renovations based on how recently they occurred.
# Assumption: More recent renovations drive the price up more.
# There are records of renovations that are more than 30 years old.
# For practicality, we will ignore this "old" renovations and only
# consider a house renovated if it was remodeled within the last 30 years.
features_df.yr_renovated.fillna(0,inplace=True)
features_df['yr_renovated'] = features_df['yr_renovated'].astype(int)
d = {range(0,2000):'no', range(2000, 2015):'recent'}

features_df['renovated'] = features_df['yr_renovated'].apply(lambda x: next((v for k, v in d.items() if x in k), 'no'))
features_df['renovated'] = features_df['renovated'].astype('category')

#drop yr_renovated from features_df
features_df.drop('yr_renovated',axis=1,inplace=True)
print('Most houses have not been remodeled recently.')

#### 12. 'yr_built' feature

In [ ]:
# Create a new feature called 'vintage' that will describe how recently
# a house was built.
d = {range(1900,1970):'Before70s',range(1970, 1980): '70s', range(1980, 1990): '80s', range(1990, 2000): '90s', range(2000, 2010): '2000s',range(2010, 2015): '2010s'}

features_df['built'] = features_df['yr_built'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))
print('done')
features_df['built'] = features_df['built'].astype('category')

#drop the year built column, since we categorized it in bins
features_df.drop('yr_built',axis=1,inplace=True)
features_df['built'].head()


#### 13. 'sqft_lot' feature:

In [ ]:
plt.boxplot(features_df.sqft_lot)
plt.show()
print ('Skewness =', skew(features_df.sqft_lot))
print ('kurtosis =', kurtosis(features_df.sqft_lot))

In [ ]:
# Create a new feature to deal with lot size
# let's check out the distribution
features_df.sqft_lot = np.log(features_df.sqft_lot)
features_df.sqft_lot.hist(density=True,color='green',alpha=0.4)

print ('Skewness =', skew(features_df.sqft_lot))
print ('kurtosis =', kurtosis(features_df.sqft_lot))




Observations: improved the kurtosis, but it's still a bit high.

#### 14. and 15. 'lat' and 'long' features

> Using the lat lon variables, approximate the distance of houses to downtown seattle.

In [ ]:
# calculate a proxy distance from seattle downtown
sea_lat  =  47.6050    # seattle's latitude in decimal degrees
sea_long = -122.3344   # seattle's longitude ''  ''      ''
lat_conv = 69.09       # approximate factor to convert decimal degrees to distance in miles
long_conv = 46.54      # gotten from http://www.csgnetwork.com/degreelenllavcalc.html

# go through latitude and longitude data and calculate component distance 
# from seattle.
features_df['sea_dist_lat'] = abs(sea_lat - features_df['lat'])*lat_conv
features_df['sea_dist_long'] = abs(sea_long - features_df['long'])*long_conv

# Using the components, calculate approximate distance using Pythagorean theorem.
features_df['approx_dist'] = np.sqrt((features_df['sea_dist_lat'])**2 + (features_df['sea_dist_long'])**2)



#### QC distance approximation

> Calculating distances using this method is a crude approximation.

> It is necessary to check whether the results make sense geographically.

> Calculations were validated by mapping the data and researching distances manually.

In [ ]:
# First QC tool: generate a histogram of distances from the city center
# to the various houses. 
features_df.approx_dist.hist(bins=50,color='green',alpha=.4)
plt.xlabel('Distance (mi)')
plt.ylabel('Count')
plt.title('Approximate Distance to Seattle Downtown')
plt.show()

In [ ]:
# Based on this histogram, when we plot a heatmap of locations, most
# houses should be located within a 10-15 radius of the city center.

#### Plot heatmap of location values

In [ ]:
# Use the folium utility to create a house density map.
# we used the original lat and long values to determine whether most
# houses are located very near Seattle.

def map_values(center_lat, center_long, df):
   
    m = folium.Map([center_lat,center_long], zoom_start=10)
    folium.Marker([center_lat,center_long]).add_to(m)

    locations = df[['lat', 'long']]
    locationlist = locations.values.tolist()
    m.add_children(plugins.HeatMap(locationlist, radius=20, min_opacity=0.25))
    return m

In [ ]:
map_values(sea_lat,sea_long,df)

In [ ]:
# Yes, after validating our data using this and outside map
# we can include this distance approximation in our analysis.

In [ ]:
# reformat / drop columns 
features_df.drop('sea_dist_long',axis=1,inplace=True)
features_df.drop('sea_dist_lat',axis=1,inplace=True)
features_df.drop('lat',axis=1,inplace=True)
features_df.drop('long',axis=1,inplace=True)
features_df.head()

#### Check data info again


In [ ]:
features_df.info()

In [ ]:
features_df.describe()

#### Create heatmaps and correlation matrix

In [ ]:
features_df.corr()>.8

####  16 , 17, 18 : 'sqft_living' , 'sqft_above' , 'sqft_living15' features

>  These variables are correlated over 70% of the time.
>  Let's choose only one for our model. The best correlated with price!

In [ ]:
# calculate correlations with target values for correlated variables
# to choose the best for our model
corr_columns = ['sqft_living','sqft_living15','sqft_above']
df_corr = price_df.join(features_df[corr_columns],how='left')
df_corr.corr()

In [ ]:
# based on these results, we will nix:  sqft_abovve

In [ ]:
features_df.drop('sqft_above',axis=1,inplace=True)
#we had dropped more features with a threshhold of .75

In [ ]:
# transform to float
features_df['sqft_living'] = features_df['sqft_living'].astype(float)

#### 19 sqft_lot15 and sqft_lot

In [ ]:
#dropped these before
#features_df.drop('sqft_lot15',axis=1,inplace=True)

In [ ]:
# Remaining correlations
features_df.corr()>.8

In [ ]:
# Looks good, no remaining strong correlations!
# for now we will keep these

In [ ]:
pd.plotting.scatter_matrix(features_df,figsize  = [9, 9], color='green',alpha=0.4)
plt.show()

#### Dummy variables


In [ ]:
features_df.info()

In [ ]:
features_df['bedrooms'] = features_df['bedrooms'].astype('float')
features_df['grade'] = features_df['grade'].astype('float')
features_df['sqft_basement'] = features_df['sqft_basement'].astype('float')
features_df['sqft_living15'] = features_df['sqft_living15'].astype('float')
features_df['sqft_lot15'] = features_df['sqft_lot15'].astype('float')

In [ ]:
#features_df = features_df.drop(cat_vars, axis=1)

#bed= pd.get_dummies(features_df['bedrooms'], prefix="bed")
#bath= pd.get_dummies(features_df['bathrooms'], prefix="bath")
floor= pd.get_dummies(features_df['floors'], prefix="flo")
water= pd.get_dummies(features_df['waterfront'], prefix="water")
#view= pd.get_dummies(features_df['view'], prefix="view")
cond = pd.get_dummies(features_df['condition'], prefix="cond")
#grade= pd.get_dummies(features_df['grade'], prefix="grade")
zipc= pd.get_dummies(features_df['zipcode'], prefix="zip")
#sold= pd.get_dummies(features_df['yr_sold'], prefix="sold")
renov = pd.get_dummies(features_df['renovated'], prefix="renov")
built = pd.get_dummies(features_df['built'], prefix="built")
#base = pd.get_dummies(features_df['basement'], prefix="base")

In [ ]:

features_df = pd.concat([features_df,floor,water,cond,renov,built,zipc],axis=1) #drop bed bath drop grade as test
features_df.shape

# drop categorical variables written before hot encoding
cat_vars = ['floors','waterfront','condition','zipcode','renovated','built']
features_df.drop(cat_vars,axis=1,inplace=True)

In [ ]:
features_df.info()

#### Normalize values

In [ ]:
# normalize continuous features in dataframe
column = 'bedrooms'
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()

column = 'bathrooms'
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()

column = 'grade'
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()

column = 'sqft_basement'
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()


column ='sqft_living'
features_df[column] = np.log(features_df[column])
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()

column ='sqft_living15'
features_df[column] = np.log(features_df[column])
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()

column ='approx_dist'
features_df[column] = np.log(features_df[column])
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()

column ='sqft_lot'
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()

column ='sqft_lot15'
features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()


In [ ]:
features_df.head()

#### Check Linearity Assumptions

In [ ]:
# check distributions for the 3 continuous variables

for column in features_df.iloc[:,0:9]:
    features_df[column].plot.hist(normed=True,color='green',alpha=0.4,label=column, range=(features_df[column].min(),features_df[column].max()))
    features_df[column].plot.kde(label='kde',color='indigo',alpha=0.5)
    plt.legend()
    plt.show()
    print ('Skewness =', skew(features_df[column]))
    print ('kurtosis =', kurtosis(features_df[column]))
    
    
    
    

In [ ]:
temp_price = price_df['price']
for column in features_df.iloc[:,0:9]:
    sns.jointplot(x=column, y=temp_price,
                  data=features_df, 
                  kind='reg', 
                  color = 'green',
                  label=column,
                  joint_kws={'line_kws':{'color':'indigo'}})
    sns.regplot(features_df[column], temp_price, label=column, color='green')
    plt.legend()
    plt.show()

#### Run first model and identify opportunities for improvement.

### Run Linear Regression for Feature Selection

In [ ]:
# returns a dataframe with regression information per variable.
# df is the dataframe containing features
# price_df is the dataframe containing the target
# pvalue is the maximum p-value accepted.

def run_single_linreg(df, price_df,pvalue):
    col_names = df.columns
    data_df = price_df.join(df,how='left')

    results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
    for val in (col_names):
        f = 'price~' + val
        model = smf.ols(formula=f, data=data_df).fit()
        X_new = pd.DataFrame({val: [data_df[val].min(), data_df[val].max()]});
        preds = model.predict(X_new)
        results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    results_df = pd.DataFrame(results, columns=results[0])
    results_df.drop(0,inplace=True)
    results_df.set_index('ind_var',inplace=True)
    results_df = results_df[results_df['p-value']<=pvalue]
    return results_df
    

In [ ]:
single_linreg_1 = run_single_linreg(features_df, price_df,0.05)
single_linreg_1.sort_values('r_squared',ascending=False).head()

### PREDICTORS: 1st Pass

In [ ]:
# features selected as 1st pass
predictors1 = list(single_linreg_1.index)
len(predictors1)

features_df = features_df[predictors1]

### Run Multiple Linear Regression Model #1

In [ ]:
# This function takes features (predictors) inside a dataframe (df)
# and creates the best fit to explain the 'price' (price_df)

def run_mult_linreg(df, price_df, predictors):
    y = price_df.price
    X = df[predictors]
    n_predictors = len(predictors)
    linreg = LinearRegression()
    r_list = []
    adj_r_list = []
    list_n = list(range(n_predictors//2,n_predictors))  #n predictors calculated above
    for n in list_n: 
        select_n = RFE(linreg, n_features_to_select = n)
        select_n = select_n.fit(X, np.ravel(y))
        selected_columns = X.columns[select_n.support_ ]
        linreg.fit(X[selected_columns],y)
        yhat = linreg.predict(X[selected_columns])
        SS_Residual = np.sum((y-yhat)**2)
        SS_Total = np.sum((y-np.mean(y))**2)
        r_squared = 1 - (float(SS_Residual))/SS_Total
        print(r_squared)
        adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
    r_list.append(r_squared)
    adj_r_list.append(adjusted_r_squared)
    print('r-squared: ' + str(r_list))
    print('adj r-squared: ' + str(adj_r_list))
    return linreg

In [ ]:
# Run the first linear regression and print r-squared values
mult_linreg1 = run_mult_linreg(features_df,price_df,predictors1)

#### Cross Validation Scores

In [ ]:
def run_cross_validation(linreg,df, price_df, predictors,cv_number):
    y = price_df.price
    X = df[predictors]
    n_predictors = len(list(predictors))
    select_n = RFE(linreg, n_features_to_select = n_predictors)
    select = select_n.fit(X, np.ravel(y))
    selected_columns = X.columns[select_n.support_]

    cv_10_results = cross_val_score(linreg, X[selected_columns], y, cv=10, scoring="neg_mean_squared_error")
    print(str(n_predictors) + ' predictors used.')
    print(cv_10_results)

In [ ]:
run_cross_validation(mult_linreg1,features_df,price_df,predictors1,10)

In [ ]:
# Large cross-validation values indicate multicollinearity issues!

In [ ]:
def run_ols_model(df, price_df,predictors):
    data_df = price_df.join(df,how='left')
    outcome = 'price'

    data_df.info()
    predictors_list = '+'.join(list(predictors))

    formula = outcome + "~" + predictors_list
    model = ols(formula=formula, data=data_df).fit()
    return(model)

In [ ]:
ols_model_1 = run_ols_model(features_df,price_df,predictors1)
ols_model_1.summary()

> Observation: There are multicollinearity issues... Need to remove binary categories or other correlated variables.

> Bedrooms is negatively correlated!

### ANALYSIS

In [ ]:
fig = sm.graphics.qqplot(ols_model_1.resid, dist=stats.norm, line=('45'), fit=True,color='green',alpha=.4)



> Variables behave linearly for about 2 standard deviations from the mean in each direction


In [ ]:
data_df1 = price_df.join(features_df,how='left')
plt.scatter(ols_model_1.predict(data_df1), ols_model_1.resid,color='green',alpha=.4)
plt.plot(ols_model_1.predict(data_df1), [0 for i in range(len(data_df1))],color='indigo',alpha=.6)
                                                      
                                                      

> Overall this is a good result.

## ITERATION 2

### Explore Variable Relationships

#### Bedrooms vs. Living Area

In [ ]:
plt.scatter(features_df.bedrooms,features_df.sqft_living,color='green',alpha=0.4)
plt.xlabel('Bedrooms (normalized)')
plt.ylabel('Living Area (sqft)')

This datapoint (33 bedrooms) looks like a typing error. Maybe they meant 3 bedrooms

In [ ]:
df.bedrooms.value_counts()

In [ ]:
# let's find out this datapoint in the dataframe
features_df.loc[df['bedrooms'] == 33]

In [ ]:
df.loc[df['bedrooms'] == 11]

In [ ]:
df.loc[df['bedrooms'] == 10]

It is clear this was a typo, a house with 1620 sqft cannot have 33 bedrooms

In [ ]:
# Let's replace this value with the log value for 3, which is the median
value = (3 - df.bedrooms.mean()) / df.bedrooms.std()
features_df.loc[2402100895,'bedrooms'] = value
features_df.loc[1773100755,'bedrooms'] = value
features_df.loc[5566100170,'bedrooms'] = value

In [ ]:
plt.scatter(features_df.bedrooms,features_df.sqft_living, color='green',alpha=0.4)
plt.xlabel('Corrected # Bedrooms')
plt.ylabel('Living Area (sqft)')

> Much better!

#### Remove Binary / Singular Categories

In [ ]:
features_df.drop('water_0',axis=1,inplace=True)  # Remove no waterfront
features_df.drop('renov_no',axis=1,inplace=True) # Remove no renovations



In [ ]:
features_df.drop('flo_1',axis=1,inplace=True)


In [ ]:
sns.heatmap(features_df.corr().iloc[0:10,0:10], center=0, cmap='Spectral');

In [ ]:
features_df.corr().iloc[0:30]>.7

In [ ]:
features_df.drop('bathrooms',axis=1,inplace=True)
features_df.drop('bedrooms',axis=1,inplace=True)

In [ ]:
features_df.drop('sqft_living15',axis=1,inplace=True)
features_df.drop('grade',axis=1,inplace=True)

In [ ]:
sns.heatmap(features_df.corr(), center=0, cmap='Spectral');

### Run Second Iteration

In [ ]:
linreg2 = run_single_linreg(features_df, price_df,0.05)
predictors2 = list(linreg2.index)
features_df = features_df[predictors2]
len(predictors2)

In [ ]:
mult_linreg2 = run_mult_linreg(features_df, price_df, predictors2)

In [ ]:
run_cross_validation(mult_linreg2,features_df,price_df,predictors2,10)

In [ ]:
ols_model_2 = run_ols_model(features_df,price_df,predictors2)
ols_model_2.summary()

In [ ]:
fig = sm.graphics.qqplot(ols_model_2.resid, dist=stats.norm, line='45', fit=True)

In [ ]:
sns.heatmap(features_df.corr(), center=0, cmap='Spectral');

### Run 3rd Iteration

In [ ]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
predictors3 = stepwise_selection(features_df, price_df)

In [ ]:
len(predictors3)

In [ ]:
### Run 3rd Iteration

In [ ]:
features_df = features_df[predictors3]
#Run the third linear regression and print r-squared values
mult_linreg3 = run_mult_linreg(features_df,price_df,predictors3)

In [ ]:
run_cross_validation(mult_linreg3,features_df,price_df,predictors3,10)

In [ ]:
ols_model_2 = run_ols_model(features_df,price_df,predictors3)
ols_model_2.summary()

In [ ]:
# Final check on correlated variables
find_corr_vars(features_df,0.7)

In [ ]:
plt.scatter(features_df.sqft_basement,price_df.price)

### QC: Train/Test MSE

In [ ]:
from sklearn.model_selection import train_test_split
X = features_df[predictors3]
y = price_df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=3)

linreg = LinearRegression()
MSE_test = []
MSE_train = []
list_n = list(range(6,86,10))
for n in list_n: 
    select_n = RFE(linreg, n_features_to_select = n)
    select_n = select_n.fit(X_train, np.ravel(y_train))
    selected_columns = X.columns[select_n.support_ ]
    linreg.fit(X_train[selected_columns],y_train)
    yhat_train = linreg.predict(X_train[selected_columns])
    yhat_test = linreg.predict(X_test[selected_columns])
    mse_train = np.sum((y_train-yhat_train)**2)/len(y_train)
    mse_test =np.sum((y_test-yhat_test)**2)/len(y_test)
    print(mse_train)
    print(mse_test)
MSE_test.append(mse_test)
MSE_train.append(mse_train)
print(n)


In [ ]:
data_df = price_df.join(features_df,how='left')
data_df.corr().loc['price'].sort_values(ascending=False)

In [ ]:
predictors2 = stepwise_selection(features_df, price_df)